In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from torchinfo import summary
from torchvision import datasets, transforms, models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


## Implementatation

In [2]:
class VGGNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.conv1 = self.create_conv_block(n_channels, 64, 2)
        self.conv2 = self.create_conv_block(64, 128, 2)
        self.conv3 = self.create_conv_block(128, 256, 3)
        self.conv4 = self.create_conv_block(256, 512, 3)
        self.conv5 = self.create_conv_block(512, 512, 3)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, n_classes),
        )
    
    def create_conv_block(self, input_channels, n_channels, n_layers):
        conv_block = nn.Sequential(
            nn.Conv2d(input_channels, n_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            *[layer for _ in range(n_layers-1) for layer in [nn.Conv2d(n_channels, n_channels, kernel_size=3, padding=1), nn.ReLU()]],
            nn.MaxPool2d(2),
        )
        return conv_block

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.classifier(x)
        return x

In [3]:
vgg_model = VGGNet(3, 1000).to('cpu')
summary(vgg_model, input_size=(1, 3, 224, 224), col_names=['input_size', 'output_size', 'num_params'], device='cpu')

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGGNet                                   [32, 3, 224, 224]         [32, 1000]                --
├─Sequential: 1-1                        [32, 3, 224, 224]         [32, 64, 112, 112]        --
│    └─Conv2d: 2-1                       [32, 3, 224, 224]         [32, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [32, 64, 224, 224]        [32, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [32, 64, 224, 224]        [32, 64, 224, 224]        36,928
│    └─ReLU: 2-4                         [32, 64, 224, 224]        [32, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [32, 64, 224, 224]        [32, 64, 112, 112]        --
├─Sequential: 1-2                        [32, 64, 112, 112]        [32, 128, 56, 56]         --
│    └─Conv2d: 2-6                       [32, 64, 112, 112]        [32, 128, 112, 112]       73,856
│    └─ReLU: 2-7        

In [4]:
vgg_torch_model = models.vgg16().to('cpu')
summary(vgg_torch_model, input_size=(1, 3, 224, 224), col_names=['input_size', 'output_size', 'num_params'], device='cpu')

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGG                                      [32, 3, 224, 224]         [32, 1000]                --
├─Sequential: 1-1                        [32, 3, 224, 224]         [32, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [32, 3, 224, 224]         [32, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [32, 64, 224, 224]        [32, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [32, 64, 224, 224]        [32, 64, 224, 224]        36,928
│    └─ReLU: 2-4                         [32, 64, 224, 224]        [32, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [32, 64, 224, 224]        [32, 64, 112, 112]        --
│    └─Conv2d: 2-6                       [32, 64, 112, 112]        [32, 128, 112, 112]       73,856
│    └─ReLU: 2-7                         [32, 128, 112, 112]       [32, 128, 112, 112]       --
│    └─Conv2d: 2-8      

In [5]:
del vgg_model

## Training

In [6]:
from pathlib import Path

TRAIN_RATIO = 0.8
data_dir = Path('./data/')

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_ds = datasets.STL10(data_dir, split='train', download=True, transform=transform)
train_ds, val_ds = random_split(train_ds, (TRAIN_RATIO, 1 - TRAIN_RATIO))
val_ds.transform = transform
test_ds = datasets.STL10(data_dir, split='test', download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
import wandb
from src.engine import *

config = dict(batch_size=16, lr=3e-3, epochs=5, dataset='STL10')
with wandb.init(project='pytorch-study', name='VGG16', config=config) as run:
    w_config = run.config
    train_dl = DataLoader(train_ds, batch_size=w_config.batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=w_config.batch_size, shuffle=True)
    
    n_classes = len(train_ds.dataset.classes)
    vgg_model = VGGNet(3, n_classes).to(DEVICE)
        
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(vgg_model.parameters(), lr=w_config.lr, weight_decay=0.3)
    
    loss_history, acc_history = train(vgg_model, train_dl, val_dl, criterion, optimizer, w_config.epochs, DEVICE, run) 

Epoch=5: 100%|██████████| 5/5 [05:26<00:00, 65.36s/it, train_loss=2.303, train_acc=10.37%, val_loss=2.303, val_acc=8.81%]
